In [706]:
import pandas as pd
import numpy as np
import umap.umap_ as umap
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import ast
from kmodes.kprototypes import KPrototypes
from kmodes.kmodes import KModes
from collections import Counter
from sklearn import impute
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.pipeline import make_pipeline, Pipeline 
from gower import gower_matrix
from utils import multiple_graphs
from prince import MCA, FAMD, MFA, PCA
from hyperopt import fmin, tpe, hp, Trials
%matplotlib inline

In [707]:
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message="using precomputed metric; inverse_transform will be unavailable")

In [708]:
data = pd.read_csv('/Users/e.kireev/Desktop/users_segmentation/Result_59.csv')
data.head(100)

,id,cls,title,geo_target,for_kids,duration,overall_duration,is_4k,devices,total_episodes,total_seasons,genres,rating_imdb,rating_age,rating_kp,rating_start,rating_total,year,popularity,popularity_kids
0,394ec71b-ee9c-49a7-ba63-dcf2bf97f50b,Movie,Лена и львенок,"['EE', 'LT', 'LV', 'AZ', 'AM', 'BY', 'KG', 'KZ...",True,5402500,0,False,"['Web', 'Mobile', 'TV']",NaN,NaN,"['family', 'comedy', 'adventure']",5.4,6,7.8,NaN,8.4,2020.0,70.0,70.0
1,4fb22f07-16d5-4659-ae03-5bae62ccc949,Series,Под прикрытием,['RU'],False,0,49706880,False,"['Web', 'Mobile', 'TV']",22.0,1.0,"['melodrama', 'comedy', 'action']",6.5,16,7.5,NaN,7.9,2022.0,81.0,NaN
2,3ddcdfb9-23fd-423b-a859-914a6e71c5d0,Movie,Подростки: Первая любовь,"['AZ', 'AM', 'BY', 'KG', 'KZ', 'MD', 'RU', 'TJ...",False,6097042,0,True,"['Web', 'Mobile', 'TV']",NaN,NaN,"['melodrama', 'music', 'youth']",5.8,18,7.2,NaN,8.2,2024.0,725.0,NaN
3,550ce943-319d-4dd4-a984-446a0317a721,Series,КрисТина,"['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO...",False,0,12328120,True,"['Web', 'Mobile', 'TV']",18.0,1.0,"['thriller', 'drama']",7.4,18,6.2,NaN,6.8,2021.0,745.0,NaN
4,5276792f-926e-440f-acc8-63739b38c64c,Movie,"Музыка, исцеляющая души",['RU'],False,714960,0,False,"['Web', 'Mobile', 'TV']",NaN,NaN,['documental'],NaN,12,NaN,NaN,NaN,2016.0,6.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,073d0610-9914-4662-b12e-04f338359bc9,Movie,Мини-вселенная,['RU'],False,2999840,0,False,"['Web', 'Mobile', 'TV']",NaN,NaN,['documental'],6.6,18,NaN,NaN,NaN,2017.0,25.0,NaN
96,b7758553-9042-463c-a3a4-eaef12916e5c,Movie,Счастье в конверте,"['AZ', 'AM', 'BY', 'KG', 'KZ', 'MD', 'RU', 'TJ...",False,5885480,0,False,['Web'],NaN,NaN,"['melodrama', 'family']",6.4,18,8.4,NaN,9.0,2020.0,1.0,NaN
97,1683de62-935b-4ac7-9c95-02083ebfd7d0,Movie,Принципиально документально,"['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO...",False,1341200,0,False,"['Web', 'Mobile', 'TV']",NaN,NaN,"['comedy', 'short']",NaN,18,7.1,NaN,6.9,2021.0,149.0,NaN
98,fa8c70f1-dd94-4bd9-8e80-60004007da4a,Movie,Доктор Лиза,"['AM', 'AZ', 'BY', 'GE', 'KZ', 'KG', 'MD', 'RU...",False,6910840,0,False,['Web'],NaN,NaN,"['drama', 'bio']",7.2,18,8.3,NaN,8.4,2020.0,1.0,NaN


In [709]:
# for col in ['devices', 'genres']:
#     data[col] = data[col].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
# data.head()

In [710]:
data.isnull().sum()

id                     0
cls                    0
title                  0
geo_target             0
for_kids               0
duration               0
overall_duration       0
is_4k                  0
devices                0
total_episodes      1017
total_seasons       1017
genres                 0
rating_imdb          289
rating_age             0
rating_kp            199
rating_start        1586
rating_total         275
year                   1
popularity             9
popularity_kids     1331
dtype: int64

In [711]:
df = data.drop(['id', 'title'], axis=1)
df.head()

,cls,geo_target,for_kids,duration,overall_duration,is_4k,devices,total_episodes,total_seasons,genres,rating_imdb,rating_age,rating_kp,rating_start,rating_total,year,popularity,popularity_kids
0,Movie,"['EE', 'LT', 'LV', 'AZ', 'AM', 'BY', 'KG', 'KZ...",True,5402500,0,False,"['Web', 'Mobile', 'TV']",NaN,NaN,"['family', 'comedy', 'adventure']",5.4,6,7.8,NaN,8.4,2020.0,70.0,70.0
1,Series,['RU'],False,0,49706880,False,"['Web', 'Mobile', 'TV']",22.0,1.0,"['melodrama', 'comedy', 'action']",6.5,16,7.5,NaN,7.9,2022.0,81.0,NaN
2,Movie,"['AZ', 'AM', 'BY', 'KG', 'KZ', 'MD', 'RU', 'TJ...",False,6097042,0,True,"['Web', 'Mobile', 'TV']",NaN,NaN,"['melodrama', 'music', 'youth']",5.8,18,7.2,NaN,8.2,2024.0,725.0,NaN
3,Series,"['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO...",False,0,12328120,True,"['Web', 'Mobile', 'TV']",18.0,1.0,"['thriller', 'drama']",7.4,18,6.2,NaN,6.8,2021.0,745.0,NaN
4,Movie,['RU'],False,714960,0,False,"['Web', 'Mobile', 'TV']",NaN,NaN,['documental'],NaN,12,NaN,NaN,NaN,2016.0,6.0,NaN


In [712]:
imp = impute.SimpleImputer(strategy = "constant", fill_value=0)
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.drop(num_cols, axis=1).columns
df[num_cols] = imp.fit_transform(df[num_cols])
df.head()

,cls,geo_target,for_kids,duration,overall_duration,is_4k,devices,total_episodes,total_seasons,genres,rating_imdb,rating_age,rating_kp,rating_start,rating_total,year,popularity,popularity_kids
0,Movie,"['EE', 'LT', 'LV', 'AZ', 'AM', 'BY', 'KG', 'KZ...",True,5402500.0,0.0,False,"['Web', 'Mobile', 'TV']",0.0,0.0,"['family', 'comedy', 'adventure']",5.4,6.0,7.8,0.0,8.4,2020.0,70.0,70.0
1,Series,['RU'],False,0.0,49706880.0,False,"['Web', 'Mobile', 'TV']",22.0,1.0,"['melodrama', 'comedy', 'action']",6.5,16.0,7.5,0.0,7.9,2022.0,81.0,0.0
2,Movie,"['AZ', 'AM', 'BY', 'KG', 'KZ', 'MD', 'RU', 'TJ...",False,6097042.0,0.0,True,"['Web', 'Mobile', 'TV']",0.0,0.0,"['melodrama', 'music', 'youth']",5.8,18.0,7.2,0.0,8.2,2024.0,725.0,0.0
3,Series,"['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO...",False,0.0,12328120.0,True,"['Web', 'Mobile', 'TV']",18.0,1.0,"['thriller', 'drama']",7.4,18.0,6.2,0.0,6.8,2021.0,745.0,0.0
4,Movie,['RU'],False,714960.0,0.0,False,"['Web', 'Mobile', 'TV']",0.0,0.0,['documental'],0.0,12.0,0.0,0.0,0.0,2016.0,6.0,0.0


In [713]:
df.isnull().sum()

cls                 0
geo_target          0
for_kids            0
duration            0
overall_duration    0
is_4k               0
devices             0
total_episodes      0
total_seasons       0
genres              0
rating_imdb         0
rating_age          0
rating_kp           0
rating_start        0
rating_total        0
year                0
popularity          0
popularity_kids     0
dtype: int64

In [714]:
cat_data = df[cat_cols]
# mca = MCA(n_components=len(cat_cols), n_iter=10, random_state=42, engine='sklearn')
mca = MCA(n_components=3, n_iter=10, random_state=42, engine='sklearn')
mca = mca.fit(cat_data)
modality_contributions = mca.column_contributions_
explained_inertia = mca.eigenvalues_

dummy_to_feat = [col.split('_')[0] for col in modality_contributions.index]

feat_importance = {}
for feat in set(dummy_to_feat):
    idx = [i for i,f in enumerate(dummy_to_feat) if f == feat]
    m = modality_contributions.iloc[idx,:].values
    imp = np.sum(np.abs(m) * explained_inertia, axis=1).sum()
    feat_importance[feat] = imp
feat_importance = pd.Series(feat_importance).sort_values(ascending=False)
print(feat_importance)

genres     0.409179
geo        0.369313
for        0.124431
devices    0.120886
cls        0.087533
is         0.055051
dtype: float64


In [715]:
df['geo_target'].apply(lambda x: len(x)).nunique()

26

Оцениваем важность числовых признаков

In [660]:
from prince import PCA
pca = PCA(n_components=len(df[num_cols].columns), rescale_with_mean=True, rescale_with_std=True, random_state=42, engine='sklearn')
pca.fit(df[num_cols])
loadings = pca.column_correlations
explained_var = pca.eigenvalues_
feature_importance = (np.abs(loadings).multiply(explained_var, axis=1)).sum(axis=1)
feature_importance = feature_importance.sort_values(ascending=False)
feature_importance

variable
duration            3.372992
overall_duration    3.333577
total_seasons       3.137302
rating_imdb         3.092071
rating_total        3.025690
total_episodes      2.953241
popularity_kids     2.803745
rating_kp           2.780348
popularity          2.621382
rating_age          2.477790
rating_start        1.865147
year                1.858949
dtype: float64

In [716]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

X_scaled = StandardScaler().fit_transform(df[num_cols])

pca = PCA()
pca.fit(X_scaled)

loadings = pd.DataFrame(pca.components_.T, index=df[num_cols].columns)


explained_var = pca.explained_variance_ratio_
np.abs(loadings).multiply(explained_var, axis=1).sum(axis=1)
feature_importance = (np.abs(loadings).multiply(explained_var, axis=1)).sum(axis=1)
feature_importance = feature_importance.sort_values(ascending=False)
feature_importance

overall_duration    0.229160
rating_imdb         0.226841
duration            0.224535
rating_total        0.222885
total_seasons       0.214339
popularity_kids     0.213858
rating_kp           0.203917
popularity          0.199402
total_episodes      0.194533
rating_age          0.182723
rating_start        0.152856
year                0.145997
dtype: float64

In [311]:
df[num_cols]

,duration,overall_duration,total_episodes,total_seasons,rating_imdb,rating_age,rating_kp,rating_start,rating_total,year,popularity,popularity_kids
0,5402500.0,0.0,0.0,0.0,5.4,6.0,7.8,0.0,8.4,2020.0,70.0,70.0
1,0.0,49706880.0,22.0,1.0,6.5,16.0,7.5,0.0,7.9,2022.0,81.0,0.0
2,6097042.0,0.0,0.0,0.0,5.8,18.0,7.2,0.0,8.2,2024.0,725.0,0.0
3,0.0,12328120.0,18.0,1.0,7.4,18.0,6.2,0.0,6.8,2021.0,745.0,0.0
4,714960.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,2016.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1603,1549080.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,2024.0,11.0,0.0
1604,3000080.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,2020.0,3.0,0.0
1605,0.0,22474240.0,9.0,1.0,0.0,18.0,8.6,0.0,8.5,2021.0,7.0,0.0
1606,5257880.0,0.0,0.0,0.0,5.6,18.0,5.9,0.0,6.1,2013.0,1.0,0.0
